In [ ]:
import datetime
from pandas import Timestamp

from ipywidgets import interact

from bokeh.models.sources import ColumnDataSource
from bokeh.plotting import figure
from bokeh.io import push_notebook, show, output_notebook

from confluent_kafka import Consumer, KafkaError

In [ ]:
output_notebook()

In [ ]:
consumer = Consumer({'bootstrap.servers': 'localhost:9092', 'group.id': 'test-group',
              'default.topic.config': {'auto.offset.reset': 'earliest'}})

consumer.subscribe(['prices'])

In [ ]:
price_figure = figure(title='Prices and binned prices', plot_height=300, 
                      plot_width=600, y_range=(0, 140), x_axis_type='datetime')
price_figure.xaxis.axis_label = 'Price timestamp'
price_figure.yaxis.axis_label = 'Price'

sec_colors = [['blue'], ['red']]
price_data = ColumnDataSource(data=dict(#color=sec_colors,
                                        xs=[[datetime.datetime(2017,1,1)], [datetime.datetime(2017,1,1)]], 
                                        ys=[[100], [100]]))
line = price_figure.multi_line(xs="xs", ys="ys", source=price_data)# , line_color="color"
handle = show(price_figure, notebook_handle=True)

xs = [[], []]
ys = [[], []]
updated_data = dict(xs=xs, ys=ys)#, color=sec_colors)
n_show = 100


In [ ]:
running = True
while running:
    msg = consumer.poll()
    if not msg.error():
        #print(f'Received message: {msg.value().decode("utf-8")}')
        dt, sec, prc = msg.value().decode("utf-8").split(',')
        dt = Timestamp(dt).to_pydatetime()
        prc = float(prc)

        # Assumes just two securities
        idx = 0 if sec == 'AAA' else 1
        xs[idx].append(dt)
        ys[idx].append(prc)
        
        updated_data['xs'] = xs = [xs[0][-n_show:], xs[1][-n_show:]]
        updated_data['ys'] = ys = [ys[0][-n_show:], ys[1][-n_show:]]
        price_data.stream(updated_data, n_show)
        
        push_notebook(handle=handle)
    elif msg.error().code() != KafkaError._PARTITION_EOF:
        print(msg.error())
        running = False
        
c.close()